In [1]:
import torch
import matplotlib.pyplot as plt
from utils.defects import S_cal
import numpy as np
import pickle
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import torch
import torch.nn as nn
import tensorboard
import time
import imageio

from utils.model import DownSampleConv
from utils.defects import theta_cal, S_cal
from kinetic_solver import KineticSolver, KineticData
from nematic_env import ActiveNematicEnv, MyCallback

from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.policies import ActorCriticCnnPolicy
from stable_baselines3.common.monitor import ResultsWriter


# 初始化物理仿真器
geo_params = {
    'N': 256,
    'Nth': 256,
    'L': 10
}
flow_params = {
    'dT': 0.3,
    'dR': 0.3,
    'alpha': -10,
    'beta': 1.0,
    'zeta': 2,
    'V0': 0.0
}
simu_params = {
    'dt': 0.0004,
    'seed': 1234,
    'inner_steps': 10,
    'outer_steps': 5000
}
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print('device = ', device)

device =  cpu


In [1]:
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np

# 设置保存图片的文件夹
output_folder = '/home/hou63/pj2/Nematic_RL/movie'
os.makedirs(output_folder, exist_ok=True)

# random np matrix 256*256*100
np.random.seed(0)
data = np.random.rand(256, 256, 100)

# 生成并保存多张图片（假设你想生成100帧的图片）
num_frames = 100
for i in range(num_frames):
    plt.figure()
    plt.imshow(data[:,:,i])  # 示例图，可替换为你的绘图逻辑
    plt.title(f"Frame {i+1}")
    plt.savefig(f"{output_folder}/frame_{i:03d}.png")
    plt.close()

# 图片文件路径按顺序排序
image_files = sorted([os.path.join(output_folder, f) for f in os.listdir(output_folder) if f.endswith('.png')])

# 使用 OpenCV 生成视频
output_video = '/home/hou63/pj2/Nematic_RL/movie/output_video.mp4'
frame_rate = 24  # 设置帧率
frame_size = None

# 读取第一张图片的尺寸，用于视频大小设置
if image_files:
    first_frame = cv2.imread(image_files[0])
    frame_size = (first_frame.shape[1], first_frame.shape[0])

# 初始化视频写入器
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_writer = cv2.VideoWriter(output_video, fourcc, frame_rate, frame_size)

# 将每张图片写入视频
for image_file in image_files:
    frame = cv2.imread(image_file)
    video_writer.write(frame)

# 释放视频写入器
video_writer.release()
print(f"视频已成功保存为 {output_video}")

视频已成功保存为 /home/hou63/pj2/Nematic_RL/movie/output_video.mp4


In [2]:
solver_paras = (geo_params, flow_params, simu_params)
# env = ActiveNematicEnv(solver_paras, device=device)

# solver_paras = (geo_params, flow_params, simu_params)

solver = KineticSolver(*solver_paras, device=device)
data_path = '/home/yuh113/pj2/Nematic_RL/datas/data_2000.pkl'
# data_path = '/home/hou63/pj2/Nematic_RL/datas/simulation_data_test.pkl'
# simulation_data = KineticData(*solver.initialize2_pytorch(seed=918), solver.simu_args)
# simulation_data = KineticData.loader(data_path)

encoder_path = '/home/yuh113/pj2/Nematic_RL/log_model/encoder_checkpoint.pth'  # 模型保存路径


# simulation_data = solver.preloop_kinetic(simulation_data, num_itr=32000)
# print(simulation_data)
# simulation_data = simulation_data.loader(data_path, device=device)
env = ActiveNematicEnv(solver_paras, solver=solver,
                        device=device,
                        data_path=data_path, intensity=10, encoder_path=encoder_path)

modal_path = '/home/yuh113/pj2/Nematic_RL/logs_ucsd/PPO_11/lights_on_model.zip'
model = PPO.load(modal_path, env=env)

print('model loaded')

 dx_  0.0390625  dth_  0.02454369260617026  dt  0.0004


KeyboardInterrupt: 

In [ ]:
simulation_data = env.simulation_data


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.